In [39]:
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model, Sequential
from keras import regularizers
from keras.regularizers import l2


from matplotlib.pyplot import imshow
%matplotlib inline

import numpy as np
import random
import scipy.misc
from scipy import ndimage
import pydot
import cv2 as cv



In [63]:
photos = 12500
height = 128
width = 128
# upload each individual photo into separate list
cat = []
dog = []

for i in range(0,photos):

    img = cv.imread('train/cat.' + str(i) + '.jpg')
    img = cv.resize(img, (height, width))
    image_data = cv.normalize(img, img, 0, 255, cv.NORM_MINMAX)


    cat.append(image_data)
    
    if i % 1000 == 0:
        print i
    
for j in range(0,photos):

    img = cv.imread('train/dog.' + str(j) + '.jpg')
    img = cv.resize(img, (height, width))
    image_data = cv.normalize(img, img, 0, 255, cv.NORM_MINMAX)
    
    
    dog.append(image_data)
    if j % 1000 == 0:
        print j
    
    
#make each list an array


data_cat = np.asarray(cat)
data_dog = np.asarray(dog)


#put those arrays together for an input vector
X_train = np.concatenate((data_cat, data_dog))

#create output vectors
y_train_cats = np.zeros((photos,1))
y_train_dogs = np.ones((photos,1))
Y_train = np.concatenate((y_train_cats, y_train_dogs))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.04, random_state=1)

In [73]:
def CatDogModel(input_shape=None, weight_decay=0.001, batch_momentum=0.9, batch_shape=None, classes=2):
    
    X_input = Input(input_shape)
    
    # Block 1
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block1_conv1', kernel_regularizer=l2(weight_decay))(X_input)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block1_conv2', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block2_conv1', kernel_regularizer=l2(weight_decay))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block2_conv2', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block3_conv1', kernel_regularizer=l2(weight_decay))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block3_conv2', kernel_regularizer=l2(weight_decay))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block3_conv3', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    
    
    #FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    x = Conv2D(128, (5,5), activation='relu', padding='same', name='block5_conv1', kernel_regularizer=l2(weight_decay))(x)
    x = layers.Dropout(0.5)(x)
    x = Conv2D(128, (1, 1), activation='relu', padding='same', name='block5_conv2', kernel_regularizer=l2(weight_decay))(x)
    x = layers.Dropout(0.5)(x)
    x = Flatten()(x)     
    x = Dense(1, activation='sigmoid', name='fcout', kernel_regularizer=l2(weight_decay))(x)
    
    model = Model(X_input, x)
    return model

In [74]:
CatDog = CatDogModel(X_train.shape[1:])
#CatDog.load_weights('CatDog_weights.h5')
CatDog.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [75]:
CatDog.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 16)      448       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 16)      2320      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 16)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 32)        4640      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 32)        9248      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 32)        0         
__________

In [77]:
CatDog.fit(X_train, y_train, epochs=10, verbose = 1, batch_size=32)

Epoch 1/10
24000/24000 [==============================] - 2251s 94ms/step - loss: 0.9181 - acc: 0.6041
Epoch 2/10
24000/24000 [==============================] - 1625s 68ms/step - loss: 0.6783 - acc: 0.7252
Epoch 3/10
24000/24000 [==============================] - 1755s 73ms/step - loss: 0.5495 - acc: 0.7915
Epoch 4/10
24000/24000 [==============================] - 1610s 67ms/step - loss: 0.4669 - acc: 0.8337
Epoch 5/10
24000/24000 [==============================] - 1610s 67ms/step - loss: 0.4214 - acc: 0.8577
Epoch 6/10
24000/24000 [==============================] - 1612s 67ms/step - loss: 0.3850 - acc: 0.8751
Epoch 7/10
24000/24000 [==============================] - 1604s 67ms/step - loss: 0.3601 - acc: 0.8895
Epoch 8/10
24000/24000 [==============================] - 1605s 67ms/step - loss: 0.3457 - acc: 0.8962
Epoch 9/10
24000/24000 [==============================] - 1604s 67ms/step - loss: 0.3334 - acc: 0.9045
Epoch 10/10
24000/24000 [==============================] - 1606s 67ms/ste

In [78]:
preds = CatDog.evaluate(x = X_test, y = y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

1000/1000 [==============================] - 25s 25ms/step
()
Loss = 0.366105397701
Test Accuracy = 0.881


In [79]:
CatDog.save_weights('CatDog_weights.h5')
#0 epochs

In [80]:
import pandas as pd
raw = pd.read_csv('TestAnswers.csv')
outputs = raw['label']
outputs = outputs.values.reshape(1000,1)

raw = []
for i in range(1,1001):

    img = cv.imread('testphotos/' + str(i) + '.jpg')
    img = cv.resize(img, (height, width))
    image_data = cv.normalize(img, img, 0, 255, cv.NORM_MINMAX)
    raw.append(img)
#make each list an array
inputs = np.asarray(raw)

#Test data separate
preds = CatDog.evaluate(x = inputs, y = outputs)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))
#0.904

1000/1000 [==============================] - 23s 23ms/step
()
Loss = 0.387106142282
Test Accuracy = 0.878
